In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
# File paths
z_usfs_th = '../data/source/usfs/S_USA.Activity_TimberHarvest.gdb.zip'
z_usfs_hft = '../data/source/usfs/S_USA.Activity_HazFuelTrt_PL.gdb.zip'
calfire_th = '../data/source/calfire/ds816/ds816.gdb'
calfire_ntmp = '../data/source/calfire/CAL_FIRE_Nonindustrial_Timber_Management_Plans_and_Notices_TA83.geojson'
# calfire_rx = ''
# calfire_calmapper = 'TKTKTK'

# Geodataframes
usfs_th_df = gpd.read_file(z_usfs_th)
usfs_hft_df = gpd.read_file(z_usfs_hft)
calfire_th_df = gpd.read_file(calfire_th)
calfire_ntmp_df = gpd.read_file(calfire_ntmp)
# calfire_rx_df = gpd.read_file(calfire_rx)
# calfire_calmapper_df = gpd.read_file(calfire_calmapper)

# TO DO
# GOTTA VERIFY RX BECAUSE PREVIOUS URL RETURNED 404



In [3]:
usfs_th_df.head()

,ADMIN_FOREST_CODE,ADMIN_REGION_CODE,ADMIN_FOREST_NAME,PROCLAIMED_FOREST_CODE,ADMIN_DISTRICT_NAME,ADMIN_DISTRICT_CODE,HOME_ORG,ACTIVITY_UNIT_ORG,SUID,FACTS_ID,...,ACCURACY,CRC_VALUE,UK,EDW_INSERT_DATE,ETL_MODIFIED,REV_DATE,GIS_ACRES,SHAPE_Length,SHAPE_Area,geometry
0,02,02,Bighorn National Forest,0202,Buffalo Ranger District,01,020201,020201,0202011028150005000,1028150005,...,NaN,CD1D25DF2038AFCE,None,2020-07-19T14:25:56+00:00,2020-07-19T14:25:56+00:00,None,107.930,0.034124,0.000049,"MULTIPOLYGON (((-107.23963 44.14363, -107.2403..."
1,02,02,Bighorn National Forest,0202,Buffalo Ranger District,01,020201,020201,0202011004250040000,1004250040,...,NaN,399B17CBB277BB83,None,2020-07-19T14:25:56+00:00,2020-07-19T14:25:56+00:00,None,31.506,0.017875,0.000014,"MULTIPOLYGON (((-106.94824 44.25094, -106.9487..."
2,02,02,Bighorn National Forest,0202,Buffalo Ranger District,01,020201,020201,0202011028350005000,1028350005,...,NaN,706CFA35E92B339B,None,2020-07-19T14:25:56+00:00,2020-07-19T14:25:56+00:00,None,18.975,0.014559,0.000009,"MULTIPOLYGON (((-107.25058 44.11861, -107.2503..."
3,02,02,Bighorn National Forest,0202,Tongue Ranger District,06,020206,020206,0202061035020012000,1035020012,...,NaN,DB658517A3C2F3A0,None,2020-07-19T14:25:56+00:00,2020-07-19T14:25:56+00:00,None,146.942,0.045419,0.000068,"MULTIPOLYGON (((-107.41356 44.72041, -107.4134..."
4,02,02,Bighorn National Forest,0202,Buffalo Ranger District,01,020201,020201,0202011006010017000,1006010017,...,NaN,D509CF3212370E45,None,2020-07-19T14:25:56+00:00,2020-07-19T14:25:56+00:00,None,18.886,0.021457,0.000009,"MULTIPOLYGON (((-107.03571 44.14790, -107.0342..."


In [4]:
usfs_hft_df

,SUID,ORG,ACTIVITY_CODE,ACTIVITY,LOCAL_QUALIFIER,ASU_NBR_UNITS,ASU_UOM,ADMIN_REGION_CODE,ADMIN_FOREST_CODE,ADMIN_DISTRICT_CODE,...,UK_HAZ,CRC_HAZ,ETL_MODIFIED_DATE_HAZ,GIS_ACRES,PURPOSE_CODE,ACT_CREATED_DATE,ACT_MODIFIED_DATE,SHAPE_Length,SHAPE_Area,geometry
0,051051142KSTS003000,051051,1160,Thinning for Hazardous Fuels Reduction,N/A,13.3,ACRES,05,10,51,...,5926026010602_051051142KSTS003000,E45EC44410D576BB,2022-01-30T01:46:21+00:00,13.287,None,2019-09-18T10:00:50+00:00,None,0.014549,0.000006,"MULTIPOLYGON (((-123.74028 41.92627, -123.7404..."
1,051051142KSTS032000,051051,1160,Thinning for Hazardous Fuels Reduction,N/A,5.7,ACRES,05,10,51,...,5926028010602_051051142KSTS032000,F3AACD59F4E390C4,2022-01-30T01:46:36+00:00,5.724,None,2019-09-18T10:01:06+00:00,None,0.009036,0.000003,"MULTIPOLYGON (((-123.73787 41.92708, -123.7378..."
2,011101A170300009000,011104,4220,Commercial Thin,N/A,15.0,ACRES,01,11,04,...,AC1419056010602_011101A170300009000,101607C97918E832,2022-01-30T01:46:36+00:00,10.571,None,2007-07-26T12:55:17+00:00,2008-07-24T16:39:19+00:00,0.016268,0.000005,"MULTIPOLYGON (((-110.18457 45.41131, -110.1844..."
3,050254F025414001044,050254,1152,Compacting/Crushing of Fuels,N/A,74.0,ACRES,05,02,54,...,5831309010602_050254F025414001044,FF6CB6A80A9E2C93,2022-01-30T01:46:36+00:00,56.117,FTF,2019-05-14T13:43:32+00:00,2020-07-20T15:09:42+00:00,0.026386,0.000022,"MULTIPOLYGON (((-116.43455 32.88356, -116.4346..."
4,080504W070000002000,080504,1152,Compacting/Crushing of Fuels,N/A,234.0,ACRES,08,05,04,...,4089806010602_080504W070000002000,ACC1D432AEB7BF1C,2022-01-30T01:46:36+00:00,234.070,None,2013-11-19T14:02:57+00:00,2013-11-19T14:02:59+00:00,0.863211,0.000089,"MULTIPOLYGON (((-82.56442 30.19417, -82.56441 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480348,060105HEADSTE60A000,060105,1154,Chipping of Fuels,N/A,50.0,ACRES,06,01,05,...,None,00000000D15B9C30,2022-01-30T02:41:50+00:00,NaN,None,2010-07-19T17:17:21+00:00,2017-10-27T10:01:11+00:00,0.000000,0.000000,None
480349,091903VS34500401001,091903,4145,Shelterwood Removal Cut (w/ leave trees) (EA/N...,N/A,20.0,ACRES,09,19,03,...,None,00000000A0C64DED,2022-01-30T02:41:50+00:00,NaN,FTM,2017-09-02T06:47:15+00:00,2020-09-30T11:06:14+00:00,0.000000,0.000000,None
480350,021209FC0103F002003,021209,1153,"Piling of Fuels, Hand or Machine",N/A,9.0,ACRES,02,12,09,...,None,00000000DCF5E70A,2022-01-30T02:41:50+00:00,NaN,None,2017-08-25T16:11:26+00:00,2017-08-25T16:18:53+00:00,0.000000,0.000000,None
480351,030903FY20NFL001000,030903,1160,Thinning for Hazardous Fuels Reduction,N/A,57.0,ACRES,03,09,03,...,None,00000000C2CB4654,2022-01-30T02:41:50+00:00,NaN,None,2020-09-17T15:01:37+00:00,2020-09-17T15:06:36+00:00,0.000000,0.000000,None


In [5]:
calfire_th_df.head()

,GIS_ACRES,REGION,THP_YEAR,THP_NUM,COUNTY,TIMBEROWNR,LANDOWNER,SILVI_1,SILVI_2,SILVI_CAT,...,APPROVED,COMPLETED,COMMENTS,SPATL_MOD,MODIFIED,HD_NUM,GLOBALID,Shape_Length,Shape_Area,geometry
0,1.605393,1,2011,132,HUM,Humboldt Redwood Co,Humboldt Redwood Co,No Harvest Area,None,No Harvest Area,...,2012-03-13T06:59:59+00:00,2015-03-07T08:00:00+00:00,None,None,None,1-11-132-HUM,{D8E22E53-7492-4D22-8F5B-CBC5F954F005},527.480246,6496.793479,"MULTIPOLYGON (((-333282.139 279924.113, -33328..."
1,0.873469,1,2011,132,HUM,Humboldt Redwood Co,Humboldt Redwood Co,Selection,None,Unevenaged Management,...,2012-03-13T06:59:59+00:00,2015-03-07T08:00:00+00:00,None,None,None,1-11-132-HUM,{A9040FD7-BE33-4C7B-ACAC-D8C8872B6EBF},279.083219,3534.802699,"MULTIPOLYGON (((-333286.837 280121.495, -33328..."
2,0.091393,1,2011,132,HUM,Humboldt Redwood Co,Humboldt Redwood Co,Selection,None,Unevenaged Management,...,2012-03-13T06:59:59+00:00,2015-03-07T08:00:00+00:00,Special Hillslope Prescription,None,None,1-11-132-HUM,{3F6D4068-0E4E-4169-928A-91AC22C78152},74.296838,369.855112,"MULTIPOLYGON (((-333021.524 279992.971, -33303..."
3,4.997377,1,2010,81,SON,Gualala Redwood Timber LLC,Gualala Redwood Timber LLC,Selection,None,Unevenaged Management,...,2011-01-26T08:00:00+00:00,2015-12-15T08:00:00+00:00,Am 11 change TO/TLO,None,None,1-10-081-SON,{1311219D-9385-4612-9BE5-C322E5D8F653},699.009999,20223.667470,"MULTIPOLYGON (((-302841.703 90246.842, -302840..."
4,2.159867,1,2010,81,SON,Gualala Redwood Timber LLC,Gualala Redwood Timber LLC,Selection,None,Unevenaged Management,...,2011-01-26T08:00:00+00:00,2015-12-15T08:00:00+00:00,Am 11 change TO/TLO,None,None,1-10-081-SON,{D28BEA61-12F8-450B-B595-9BC7AE6E4D75},1048.547017,8740.672216,"MULTIPOLYGON (((-302941.735 90074.162, -302933..."


In [7]:
calfire_ntmp_df.head()

,OBJECTID,GIS_ACRES,REGION,NTMP_YEAR,NTMP_NUM,COUNTY,TIMBEROWNR,LANDOWNER,SILVI_1,SILVI_2,...,APPROVED,CANCELLED,COMMENTS,SPATL_MOD,MODIFIED,HD_NUM,GLOBALID,SHAPE_Length,SHAPE_Area,geometry
0,1,1.741343,1,2018,1,MEN,"Maureen McSherri, Arthur Haschak, Mara Welker","Maureen McSherri, Arthur Haschak, Mara Welker",No Harvest Area,None,...,2018-05-17T06:59:59+00:00,None,Non Timberland,None,None,1-18NTMP-001-MEN,{1A790651-5EBB-45C8-82F4-B6900487800F},0.004057,7.365471e-07,"MULTIPOLYGON (((-123.40533 39.36725, -123.4060..."
1,2,32.208667,1,2015,3,MEN,Senger Dumars Trust,"Senger Dumars Trust, Ed Barnett",Selection,None,...,2016-02-19T08:00:00+00:00,None,None,None,None,1-15NTMP-003-MEN,{08E35BAE-38E3-4F1F-85DF-0CF1B97F84FF},0.015437,1.367898e-05,"MULTIPOLYGON (((-123.76021 39.65556, -123.7597..."
2,3,7.814303,1,2015,3,MEN,Senger Dumars Trust,"Senger Dumars Trust, Ed Barnett",Selection,None,...,2016-02-19T08:00:00+00:00,None,None,None,None,1-15NTMP-003-MEN,{10851BA0-CD74-4C4D-9567-2C831EEF88F1},0.007379,3.318697e-06,"MULTIPOLYGON (((-123.76107 39.65428, -123.7610..."
3,4,1.256814,1,2018,1,MEN,"Maureen McSherri, Arthur Haschak, Mara Welker","Maureen McSherri, Arthur Haschak, Mara Welker",No Harvest Area,None,...,2018-05-17T06:59:59+00:00,None,Non Timberland,None,None,1-18NTMP-001-MEN,{8167746D-C8A3-4943-AA59-B5FB3B293E9B},0.003200,5.315777e-07,"MULTIPOLYGON (((-123.40765 39.36419, -123.4074..."
4,5,20.567624,4,2002,4,ELD,Jules Levaggi,Jules Levaggi,No Harvest Area,None,...,2002-09-30T06:59:59+00:00,None,None,None,None,4-02NTMP-004-ELD,{951CD374-B654-46BC-8711-34469A50ABB2},0.016559,8.603353e-06,"MULTIPOLYGON (((-120.60673 38.56497, -120.6072..."
